In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import precision_recall_curve, f1_score
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsRegressor

In [2]:
#reading in data
df = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2020/2020-01-21/spotify_songs.csv')

In [3]:
#cleaning

#removing nulls
df = df[df['track_name'].notnull()]

#removing playlist variables
df = df.drop(columns = ['playlist_name', 'playlist_id', 'playlist_genre', 'playlist_subgenre'])

# Filter to only unique track_ids
df = df.drop_duplicates(subset=['track_id'])

# Remove the 1 row where tempo is 0 (a song can't have a tempo of 0)
df = df[df['tempo'] != 0]

In [4]:
# split into train and test
X = df[['track_artist','danceability','energy','key','loudness','mode','speechiness','acousticness','instrumentalness','liveness','valence','tempo','duration_ms']]
y = df[['track_popularity']]

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y)

In [5]:
#preprocess data
num_col = ['danceability','energy','key','loudness','mode','speechiness','acousticness','instrumentalness','liveness','valence','tempo','duration_ms']
cat_col = ['track_artist']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('imputer',SimpleImputer(strategy='mean')),
            ('poly_features',PolynomialFeatures(degree=2, include_bias=False)),
            ('scaler',StandardScaler())
            ]), num_col), 
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_col)
    ]
)

In [ ]:
# Decision Tree
dt = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', DecisionTreeClassifier(class_weight='balanced'))
])

dt.fit(Xtrain, ytrain)


In [ ]:
y_pred_train_dt = dt.predict(Xtrain)
f1_train = f1_score(ytrain, y_pred_train_dt, average='weighted')

y_pred_test_dt = dt.predict(Xtest)
f1_test = f1_score(ytest, y_pred_test_dt, average='weighted')

print(f"F1 train score: {f1_train}")
print(f"F1 test score: {f1_test}")
#way overfit:(

F1 train score: 0.9809857217793547
F1 test score: 0.028859714372729037


In [ ]:
# linear regression
lm = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('linear_regression',LinearRegression())
])

#lm = LinearRegression()
lm.fit(Xtrain,ytrain)
y_pred_train_lm = lm.predict(Xtrain)
y_pred_test_lm = lm.predict(Xtest)

mse_train = mean_squared_error(ytrain,y_pred_train_lm)
mse_test = mean_squared_error(ytest,y_pred_test_lm)

print(f"MSE Train: {mse_train}")
print(f"MSE Test: {mse_test}")

MSE Train: 234.6060410755242
MSE Test: 472.86900157171357


In [ ]:
knn = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('knn',KNeighborsRegressor())
])

knn.fit(Xtrain,ytrain)
y_pred_train_knn = knn.predict(Xtrain)
y_pred_test_knn = knn.predict(Xtest)

In [ ]:
mse_train = mean_squared_error(ytrain,y_pred_train_knn)
mse_test = mean_squared_error(ytest,y_pred_test_knn)

print(f"MSE Train: {mse_train}")
print(f"MSE Test: {mse_test}")

In [3]:
df

,track_id,track_name,track_artist,track_popularity,track_album_id,track_album_name,track_album_release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,6f807x0ima9a1j3VPbc7VN,I Don't Care (with Justin Bieber) - Loud Luxur...,Ed Sheeran,66,2oCs0DGTsRO98Gh5ZSl2Cx,I Don't Care (with Justin Bieber) [Loud Luxury...,2019-06-14,0.748,0.916,6,-2.634,1,0.0583,0.102000,0.000000,0.0653,0.5180,122.036,194754
1,0r7CVbZTWZgbTCYdfa2P31,Memories - Dillon Francis Remix,Maroon 5,67,63rPSO264uRjW1X5E6cWv6,Memories (Dillon Francis Remix),2019-12-13,0.726,0.815,11,-4.969,1,0.0373,0.072400,0.004210,0.3570,0.6930,99.972,162600
2,1z1Hg7Vb0AhHDiEmnDE79l,All the Time - Don Diablo Remix,Zara Larsson,70,1HoSmj2eLcsrR0vE9gThr4,All the Time (Don Diablo Remix),2019-07-05,0.675,0.931,1,-3.432,0,0.0742,0.079400,0.000023,0.1100,0.6130,124.008,176616
3,75FpbthrwQmzHlBJLuGdC7,Call You Mine - Keanu Silva Remix,The Chainsmokers,60,1nqYsOef1yKKuGOVchbsk6,Call You Mine - The Remixes,2019-07-19,0.718,0.930,7,-3.778,1,0.1020,0.028700,0.000009,0.2040,0.2770,121.956,169093
4,1e8PAfcKUYoKkxPhrHqw4x,Someone You Loved - Future Humans Remix,Lewis Capaldi,69,7m7vv9wlQ4i0LFuJiE2zsQ,Someone You Loved (Future Humans Remix),2019-03-05,0.650,0.833,1,-4.672,1,0.0359,0.080300,0.000000,0.0833,0.7250,123.976,189052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32828,7bxnKAamR3snQ1VGLuVfC1,City Of Lights - Official Radio Edit,Lush & Simon,42,2azRoBBWEEEYhqV6sb7JrT,City Of Lights (Vocal Mix),2014-04-28,0.428,0.922,2,-1.814,1,0.0936,0.076600,0.000000,0.0668,0.2100,128.170,204375
32829,5Aevni09Em4575077nkWHz,Closer - Sultan & Ned Shepard Remix,Tegan and Sara,20,6kD6KLxj7s8eCE3ABvAyf5,Closer Remixed,2013-03-08,0.522,0.786,0,-4.462,1,0.0420,0.001710,0.004270,0.3750,0.4000,128.041,353120
32830,7ImMqPP3Q1yfUHvsdn7wEo,Sweet Surrender - Radio Edit,Starkillers,14,0ltWNSY9JgxoIZO4VzuCa6,Sweet Surrender (Radio Edit),2014-04-21,0.529,0.821,6,-4.899,0,0.0481,0.108000,0.000001,0.1500,0.4360,127.989,210112
32831,2m69mhnfQ1Oq6lGtXuYhgX,Only For You - Maor Levi Remix,Mat Zo,15,1fGrOkHnHJcStl14zNx8Jy,Only For You (Remixes),2014-01-01,0.626,0.888,2,-3.361,1,0.1090,0.007920,0.127000,0.3430,0.3080,128.008,367432
